In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

1 . To import the table (Scrape) from  Wikipedia page and build Dataframe.
 

In [ ]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
print(f'The number of (rows,cols) for [df] : {df.shape} ')

2. Ignore cells with a borough that is Not assigned.

In [ ]:
df1 = df[df['Borough'] != 'Not assigned']

postcodes = df1['Postcode'].unique()
print(f'The number of (rows,cols) for [df1] : {df1.shape} ')

3. To combine all neighborhoods for the same postcode in same row separated with comma.

In [ ]:
df_cleaned = pd.DataFrame({'Postcode':postcodes,
                     'Borough':[df1[df1['Postcode']==x].iloc[0,1]for x in postcodes ]})
post_list = []
postcode_joined = None
for postcode in postcodes:
    df_cond = df1[df1['Postcode'] == postcode]
    # if neighborhood is  "Not assigned" then change it to be same as borough. 
    if str(df_cond.iloc[0,2]) == 'Not assigned':
        df1[df1['Postcode'] == postcode].iloc[0,2] = df_cond.iloc[0,1]
        
    length = len(df1[df1['Postcode'] == postcode])
    for val in range(0,length):
        
        if val == 0:
            postcode_joined = str(df1[df1['Postcode'] == postcode].iloc[0,2])
        else:
            postcode_joined += ', '+ str(df1[df1['Postcode'] == postcode].iloc[val,2])
    post_list.append(postcode_joined)

df_cleaned['Neighbourhood'] = post_list              
print(f'The number of (rows,cols) for [df_cleaned] : {df_cleaned.shape} ')


Read the locations csv file and build dataframe.


In [ ]:
locations = pd.read_csv('http://cocl.us/Geospatial_data')

locations.head()

Add 2 new columns [Latitude, Longitude] from location to df_cleaned dataframe.

In [ ]:
df_cleaned['Latitude']= [locations[locations['Postal Code'] == x].iloc[0,1] for x in postcodes]
df_cleaned['Longitude']= [locations[locations['Postal Code'] == x].iloc[0,2] for x in postcodes]
df_cleaned.head()



In [ ]:
print(f'The number of (rows,cols) for [df_cleaned] : {df_cleaned.shape} ')




To get the geographical coordinates of Torrento

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))




Create a map of Toronto with neighborhoods superimposed on top

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_cleaned['Latitude'], 
                                           df_cleaned['Longitude'],
                                           df_cleaned['Borough'],
                                           df_cleaned['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
toronto_data = df_cleaned[df_cleaned['Borough'] == 'North York'].reset_index(drop=True)
toronto_data.head()

Define Foursquare Credentials and Version.

In [ ]:
CLIENT_ID = '5T2GGWMMZXTOSAYESR0ZQHUT0PPPY31ZGXMZBIKNCDBALKR3' # your Foursquare ID
CLIENT_SECRET = 'FDPDARQ4AZD3XAV235TPLHGGGUKUMLQOUAMTFOJVIXDOHSAM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)




x

In [ ]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_cleaned.loc[1,'Latitude'], 
    df_cleaned.loc[1,'Longitude'], 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()
results


create a function to find venues for each Neighbourhood.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=' \
              '{}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=df_cleaned['Neighbourhood'],
                                   latitudes=df_cleaned['Latitude'],
                                   longitudes=df_cleaned['Longitude']
                                  )



Let's check the size of the resulting dataframe

In [ ]:

print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()


Find out how many unique categories can be curated from all the returned venues

In [ ]:
# print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))



Analyze Each Neighborhood.

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [col for col in toronto_onehot.columns if col != 'Neighborhood' ]
fixed_columns =  ['Neighborhood'] + fixed_columns        
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


Grouping rows by neighborhood.

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped


Print each neighborhood


In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


Write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


Cluster Neighborhoods.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


In [ ]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data.drop('Postcode',1)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged = toronto_merged.drop([16])
toronto_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],
                                  toronto_merged['Cluster Labels'].astype(int)):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

        
map_clusters
